<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Final Assignment</font></h1>

## Introduction


In this notebook, an image classifier is created using the VGG16 pre-trained model to predict whether the images in the test data are images of cracked concrete or not. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [0]:
!wget  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-02-05 21:24:57--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  34.9MB/s    in 7.7s    

2020-02-05 21:25:05 (32.3 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [0]:
!unzip concrete_data_week4.zip

The data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, the VGG16 pre-trained model is used to create an image classifier. The model <code>VGG16</code> can be imported from <code>keras.applications</code>.

Steps to follow:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

In [0]:
import keras
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input


Using TensorFlow backend.


In [0]:
from keras.models import Sequential 
from keras.layers import Dense

In [0]:
 from keras.preprocessing.image import ImageDataGenerator

Prepare data

In [0]:
image_resize = 224
batch_size = 100
directory_training = 'concrete_data_week4/train'
directory_validation ='concrete_data_week4/valid'

In [0]:
image_generator = ImageDataGenerator(
preprocessing_function = preprocess_input
)

In [0]:
image_generator_training = image_generator.flow_from_directory( directory_training,
class_mode='categorical',
seed=24,
target_size=(image_resize, image_resize),
batch_size=batch_size)

Found 30001 images belonging to 2 classes.


In [0]:
image_generator_validation = image_generator.flow_from_directory( directory_validation,
class_mode='categorical',
seed=24,
target_size=(image_resize, image_resize),
batch_size=batch_size)

Found 9501 images belonging to 2 classes.


In [0]:
ls concrete_data_week4/valid/negative | wc

   4750    4750   47500


Create model

In [0]:
model=Sequential()
model.add(VGG16( 
    include_top = False, pooling ='avg', weights='imagenet'
    ))

In [0]:
model.add(Dense(2, activation='softmax'))


In [0]:
 model.layers

In [0]:
model.layers[0].layers

In [0]:
 model.layers[0].trainable=False

In [0]:
 model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


Compile and train

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
steps_per_epoch_training = len(image_generator_training)
steps_per_epoch_validation = len(image_generator_validation)
num_epochs = 2

In [0]:
steps_per_epoch_training

301

In [0]:
steps_per_epoch_validation

96

In [0]:
fit_history = model.fit_generator(
image_generator_training,
epochs = num_epochs,
validation_data = image_generator_validation,
validation_steps = steps_per_epoch_validation,
verbose=1
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
301/301 [==============================] - 13267s 44s/step - loss: 0.0912 - acc: 0.9693 - val_loss: 0.0243 - val_acc: 0.9941
Epoch 2/2
301/301 [==============================] - 13292s 44s/step - loss: 0.0193 - acc: 0.9958 - val_loss: 0.0144 - val_acc: 0.9964


  Save model

In [0]:
 model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

Evaluate the deep learning model on a test data.

1. Load the saved model that was built using the VGG16 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate the model on the test data, by passing the above ImageDataGenerator as an argument. 
4. Print the performance of the classifier using the VGG16 pre-trained model.


In [0]:
from keras.models import load_model

In [0]:
model_vgg16 = load_model('classifier_vgg16_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [0]:
directory_test = 'concrete_data_week4/test'

In [0]:
image_generator_test = image_generator.flow_from_directory( directory_test,
target_size=(image_resize, image_resize),
shuffle=False)

Found 500 images belonging to 2 classes.


In [0]:
model_vgg16.metrics_names

['loss', 'acc']

In [0]:
results_vgg16 = model_vgg16.evaluate_generator(image_generator_test)

In [0]:
loss_vgg16, acc_vgg16 = results_vgg16
print("VGG16 loss: ", loss_vgg16)
print("VGG16 accuracy: ", acc_vgg16)

VGG16 loss:  0.01456501765921712
VGG16 accuracy:  0.996


## Part 3

Predict whether the images in the test data are images of cracked concrete or not. 

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

In [0]:
first_batch = image_generator_test.next()

In [0]:
first_batch_labels = first_batch[1]
first_batch_labels[0:5]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [0]:
predictions_vgg16  = model_vgg16.predict_generator(image_generator_test)

In [0]:
import numpy as np

In [0]:
first_five_vgg16 = predictions_vgg16[0:5]
first_five_vgg16_index = first_five_vgg16.argmax(axis=1)

for first_five_vgg16_index in first_five_resnet_index:
  if(class_index==0):
    print('Negative')
  else: 
    print('Positive')

Positive
Positive
Positive
Positive
Positive


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).